# Standardized Data With Timestamps Suitable For Backtesting

### [Data Dictionary](https://www.calcbench.com/home/standardizedmetrics)
### `date_reported` is the timestamp when Calcbench published the data and users would have had access ot it.
### `revision_number` greater than 0 indicates that the filer reported a different value for this fact that was previously reported.

In [ ]:
import calcbench as cb
import pandas as pd
import datetime
import os
from tqdm import tqdm, tqdm_notebook
pit_columns = ['CIK', 'calendar_period', 'calendar_year', 'date_reported', 'period_start', 'period_end',
               'fiscal_period', 'fiscal_year', 'metric', 'revision_number','ticker', 'value']

In [ ]:
tickers = cb.companies(entire_universe=True).ticker

## Get Historic Data
### Takes ~12 hours

In [ ]:
output_file_name = r"C:/Users/Andrew Kittredge/documents/calcbench_data.csv"
try:
    os.remove(output_file_name)
except OSError:
    pass
for ticker in tqdm_notebook(tickers):
    try:
        pit_data = cb.point_in_time(all_face=True, all_footnotes=True, company_identifiers=[ticker], all_history=True)
    except KeyboardInterrupt:
        raise
    except Exception as e:
        tqdm.write(f"Exception getting {ticker} {e}")
    else:
        if not pit_data.empty:
            with open(output_file_name, 'a') as f:
                pit_data.reindex(columns=pit_columns).to_csv(f, header=not f.tell())


## Updates for New Filings
Run the below code in a daemon.

Calcbench pushes messages onto the queue when data is available, typically a few minutes after the SEC publishes the data.

Messages will remain in the queue for seven days, if you process goes down you will receive the messages when it is started again.

If the handle_filing function throws an error the message will be pushed back on the queue to re-try.


In [ ]:
connection_string = "Calcbench will give this to you"
subscription = "Calcbench will give this to you"

def handle_filing(filing):
    if filing["filing_type"] != "annualQuarterlyReport":
        return
    accession_id = filing['calcbench_id']
    filing_data = cb.point_in_time(accession_id=accession_id, all_face=True, all_footnotes=True)
    file_name = f"{filing['ticker']}_{filing['calcbench_finished_load'].date()}.csv"
    filing_data.reindex(columns=pit_columns).to_csv(file_name)
    
cb.handle_filings(handler=handle_filing, 
                  connection_string=connection_string,
                  subscription_name=subscription)